In [1]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from tqdm import tqdm
import tensorflow as tf

In [2]:
a1=pd.read_csv('testing.csv')
a2=pd.read_csv('trainning.csv')

In [3]:
accept1=[]
accept2=[]

for key in a1.keys():
    if len(np.unique(a1[key]))>1:
        accept1.append(key)
for key in a2.keys():
    if len(np.unique(a2[key]))>1:
        accept2.append(key) 


In [4]:
def normarlize(length):
    return (length-length.mean())/length.std(),length.mean(),length.std()

In [6]:
b1=list()
b2=list()
des=dict()
accept2=np.delete(accept2,np.argwhere(accept2=='label'))
for key in tqdm(sorted(accept2)):
    if a2[key].dtype=='int64' or a2[key].dtype=='float64':
        nm,mean,std=normarlize(a2[key])
        nm=np.expand_dims(nm.tolist(),0).T
        des[key]=[mean,std]
    elif a2[key].dtype=='O':
        uni=a2[key].unique()
        l=len(uni)
        if '-1' in uni:
            l-=1
        nm=np.zeros((len(a2[key]),l))
        uni=np.delete(uni,np.argwhere(uni=='-1'))
        uni=dict(zip(uni,list(range(len(uni)))))
        des[key]=uni
        for i,n in enumerate(a2[key]):
            try:
                nm[i,uni[n]]=1
            except:
                continue  
    else:
        continue
    b2.append(nm)    
    
for key in tqdm(sorted(accept2)):
    if a1[key].dtype=='int64' or a1[key].dtype=='float64':
        nm=(a1[key]-des[key][0])/des[key][1]
        nm=np.expand_dims(nm.tolist(),0).T
    elif a1[key].dtype=='O':
        nm=np.zeros((len(a1[key]),len(des[key].keys())))
        for i,n in enumerate(a1[key]):
            try:
                nm[i,des[key][n]]=1
            except:
                continue  
    else:
        continue
    b1.append(nm)  

100%|██████████| 45/45 [00:00<00:00, 296.57it/s]


In [7]:
data_train=np.concatenate(b2,axis=1)
data_test=np.concatenate(b1,axis=1)

In [12]:
svclassifier =SVC(kernel='linear',verbose=True,C=5)
svclassifier.fit(data_train,a2.label)

[LibSVM]

SVC(C=5, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=True)

In [13]:
pred=svclassifier.predict(data_train)
print(classification_report(a2.label, pred))
pred=svclassifier.predict(data_test)
print(classification_report(a1.label, pred))

              precision    recall  f1-score   support

           0       0.84      1.00      0.91     50400
           1       1.00      0.43      0.60     17545

   micro avg       0.85      0.85      0.85     67945
   macro avg       0.92      0.72      0.76     67945
weighted avg       0.88      0.85      0.83     67945

              precision    recall  f1-score   support

           0       0.83      1.00      0.91     21600
           1       1.00      0.43      0.60      7520

   micro avg       0.85      0.85      0.85     29120
   macro avg       0.92      0.71      0.75     29120
weighted avg       0.88      0.85      0.83     29120



In [8]:
batch_size = 1000
layer_ids = ['hidden1','hidden2','out']
layer_sizes = [data_train.shape[1], 200,150, 2]
train_inputs = tf.placeholder(tf.float32, shape=[None, layer_sizes[0]], name='train_inputs')
train_labels = tf.placeholder(tf.int32, shape=[None], name='train_labels')
for idx, lid in enumerate(layer_ids):
    with tf.variable_scope(lid):
        w = tf.get_variable('weights',shape=[layer_sizes[idx], layer_sizes[idx+1]],
                            initializer=tf.truncated_normal_initializer(stddev=0.05))
        b = tf.get_variable('bias',shape= [layer_sizes[idx+1]],
                            initializer=tf.random_uniform_initializer(-0.1,0.1))
h = train_inputs
for lid in layer_ids:
    with tf.variable_scope(lid,reuse=True):
        w, b = tf.get_variable('weights'), tf.get_variable('bias')
        if lid != 'out':
            h = tf.nn.relu(tf.matmul(h,w)+b,name=lid+'_output')
        else:
            h = tf.nn.xw_plus_b(h,w,b,name=lid+'_output')
tf_predictions = tf.nn.softmax(h, name='predictions')
tf_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf.one_hot(train_labels,depth=2), logits=h),name='loss')
tf_learning_rate = tf.placeholder(tf.float32, shape=None, name='learning_rate')
optimizer = tf.train.AdamOptimizer(learning_rate=0.001,).minimize(tf_loss)
sess=tf.Session()
sess.run(tf.global_variables_initializer())

Load model fail


In [ ]:
step=0
saver=tf.train.Saver(max_to_keep=1)
try:
    checkpoint = tf.train.latest_checkpoint('checkpoint')
    saver.restore(sess, checkpoint)
    step=int(checkpoint.split('-')[-1])+1
    print('Load complete at step: ',str(step))
except Exception:
    print('Load model fail')
    step=0
for epoch in range(step,2000):
    L=0
    for iters in range(data_train.shape[0]//batch_size):
        _,l=sess.run([optimizer,tf_loss],feed_dict={train_inputs:data_train[iters*batch_size:(iters+1)*batch_size],train_labels:a2.label[iters*batch_size:(iters+1)*batch_size]})
        L+=l
    print('Step: {:d} loss: {:f} '.format(epoch,L))
    if (epoch+1)%10==0:
        saver.save(sess,'checkpoint/model',global_step=epoch)
        count=0
        for iters in range(data_test.shape[0]//batch_size):
            predict=sess.run(tf_predictions,feed_dict={train_inputs:data_test[iters*batch_size:(iters+1)*batch_size]})
            predict=np.argmax(predict,axis=1)
            count+=sum(predict==a1.label[iters*batch_size:(iters+1)*batch_size])
        print('Accuracy: ',str(count/len(data_test)))

In [15]:
count=0
for iters in range(data_test.shape[0]//batch_size):
    predict=sess.run(tf_predictions,feed_dict={train_inputs:np.reshape(data_test[iters*batch_size:(iters+1)*batch_size],[batch_size,-1])})
    predict=np.argmax(predict,axis=1)
    count+=sum(predict==a1.label[iters*batch_size:(iters+1)*batch_size])

In [18]:
predict=sess.run(tf_predictions,feed_dict={train_inputs:data_train})
predict=np.argmax(predict,axis=1)
print(classification_report(a2.label, predict))
predict=sess.run(tf_predictions,feed_dict={train_inputs:data_test})
predict=np.argmax(predict,axis=1)
print(classification_report(a1.label, predict))

              precision    recall  f1-score   support

           0       0.87      1.00      0.93     50400
           1       1.00      0.57      0.72     17545

   micro avg       0.89      0.89      0.89     67945
   macro avg       0.93      0.78      0.83     67945
weighted avg       0.90      0.89      0.88     67945

              precision    recall  f1-score   support

           0       0.87      1.00      0.93     21600
           1       0.99      0.56      0.71      7520

   micro avg       0.88      0.88      0.88     29120
   macro avg       0.93      0.78      0.82     29120
weighted avg       0.90      0.88      0.87     29120

